## Setup a classification experiment

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('hmeq.csv')

In [3]:
# Nominal features
# Replacement using majority class
# majority class in case of JOB variable is Other
# majority class in case of REASON varibale is DebtCon

df["REASON"].fillna(value = "DebtCon",inplace = True)
df["JOB"].fillna(value = "Other",inplace = True)
df["DEROG"].fillna(value=0,inplace=True)
df["DELINQ"].fillna(value=0,inplace=True)

In [4]:
# Numeric features
# Replacement using mean of each class

df.fillna(value=df.mean(),inplace=True)

In [5]:
X = df.drop(columns=["BAD","JOB","REASON"])
y = df["BAD"]

In [6]:
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [7]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

<!-- http://127.0.0.1:7174/2783924847376/ -->

## Train the Explainable Boosting Machine (EBM)

In [8]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingClassifier(feature_names=['LOAN', 'MORTDUE', 'VALUE', 'YOJ',
                                             'DEROG', 'DELINQ', 'CLAGE', 'NINQ',
                                             'CLNO', 'DEBTINC'],
                              feature_types=['continuous', 'continuous',
                                             'continuous', 'continuous',
                                             'continuous', 'continuous',
                                             'continuous', 'continuous',
                                             'continuous', 'continuous'],
                              random_state=1)

## Global Explanations: What the model learned overall

In [9]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7174/2784009146080/ -->

- DEBTINC: Debt-to-income ratio
- DELINQ: Number of delinquent credit lines (Legally, a line of credit becomes delinquent when a borrower does not make the minimum required payments 30 to 60 days past the day on which the payments were due)
- CLAGE: Age of oldest trade line in months
- DEROG: Number of major derogatory reports
- YOJ: Years at present job
- MORTDUE: Amount due on existing mortgage
- VALUE: Value of current property
- CLNO: Number of credit lines
- LOAN: Amount of the loan request
- NINQ: Number of recent credit lines
- REASON: DebtCon = debt consolidation HomeImp = home improvement
- JOB: Six occupational categories

- BAD: 1 = client defaulted on loan 0 = loan repaid

## Local Explanations: How an individual prediction was made

In [10]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7174/2784008241360/ -->

## Evaluate EBM performance

In [11]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7174/2784013077568/ -->

## Let's test out a few other Explainable Models

In [12]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l1', solver='liblinear')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

## Compare performance using the Dashboard

In [13]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

<!-- http://127.0.0.1:7174/2783958154016/ -->

<!-- http://127.0.0.1:7174/2784012593904/ -->

<!-- http://127.0.0.1:7174/2784013077568/ -->

## Glassbox: All of our models have global and local explanations

In [14]:
lr_global = lr.explain_global(name='Logistic Regression')
tree_global = tree.explain_global(name='Classification Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

<!-- http://127.0.0.1:7174/2784013452480/ -->

<!-- http://127.0.0.1:7174/2784013211104/ -->

<!-- http://127.0.0.1:7174/2784009146080/ -->

## Dashboard: look at everything at once

In [15]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([hist, lr_global, lr_perf, tree_global, tree_perf, ebm_global, ebm_perf], share_tables=True)

<!-- http://127.0.0.1:7174/2784013429120/ -->
 Open in new window